In [358]:
%install_ext http://raw.github.com/jrjohansson/version_information/master/version_information.py
%load_ext version_information
%reload_ext version_information
%version_information numpy, scipy, matplotlib, pandas

Installed version_information.py. To use it, type:
  %load_ext version_information
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 2.7.10 64bit [GCC 4.2.1 (Apple Inc. build 5666) (dot 3)]
IPython 4.0.0
OS Darwin 14.5.0 x86_64 i386 64bit
numpy 1.9.2
scipy 0.16.0
matplotlib 1.4.3
pandas 0.16.2
Mon Sep 14 16:00:56 2015 PDT

In [359]:
!pwd

/Users/berkeleyalmand/repos/ski_in_solitude


[autoreload of version_information failed: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named version_information
]


In [360]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder as OneHotEncoder
import datetime
import seaborn as sns

%load_ext autoreload
%autoreload 2
from functions import *
from plots import *

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8  # plotsize 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import traffic data

In [412]:
df_traf = pd.DataFrame.from_csv('data/berthoud_skiers.csv')

In [413]:
df_traf.head()

,berthoud_toward_i70,berthoud_away_i70,day_of_week,skiers
FormattedDate,,,,
2014-10-31,1803,2822,4,0.971232
2014-11-01,1938,2686,5,1.169169
2014-11-02,2679,1379,6,1.314465
2014-11-03,1539,1290,0,1.061945
2014-11-04,1624,1529,1,1.013577


In [414]:
resamp_time = '1440Min'
df_all = pd.concat([df_traf.resample(resamp_time), df_met.resample(resamp_time)], axis=1).dropna()

In [415]:
df_all.head()

,berthoud_toward_i70,berthoud_away_i70,day_of_week,skiers,Time,Visibility,freezing_fog,heavy_snow,light_snow,mist,...,broken_sky,scattered_sky,overcast,snowfall,high_temp_C,low_temp_C,max_windspeed,mean_windspeed,max_rh,mean_rh
FormattedDate,,,,,,,,,,,,,,,,,,,,,
2014-11-01,1938,2686,5,1.169169,1179.805556,10.000000,0.000000,0.000000,0.000000,0.000000,...,0.013889,0.138889,0.000000,0.00,5,-1,37,18.270270,96,88.430556
2014-11-02,2679,1379,6,1.314465,1211.217391,8.623188,0.043478,0.014493,0.028986,0.072464,...,0.086957,0.289855,0.115942,0.06,-1,-6,36,15.444444,96,86.623188
2014-11-03,1539,1290,0,1.061945,1243.306452,2.281250,0.370968,0.048387,0.096774,0.080645,...,0.096774,0.080645,0.725806,0.14,-6,-11,33,16.163265,92,85.919355
2014-11-04,1624,1529,1,1.013577,1179.819444,9.425000,0.000000,0.000000,0.000000,0.013889,...,0.000000,0.000000,0.291667,0.45,-5,-13,44,19.014286,96,83.986111
2014-11-05,1672,1667,2,0.995124,1179.875000,9.013889,0.055556,0.000000,0.013889,0.055556,...,0.041667,0.055556,0.208333,0.00,-1,-10,43,19.628571,92,84.708333


In [416]:
df_all_corr = df_all.drop(['morning_away', 'evening_away', 'morning_to', 'evening_to', 'Time', 'WeatherType'], axis = 1)

ValueError: labels ['morning_away' 'evening_away' 'morning_to' 'evening_to' 'WeatherType'] not contained in axis

In [ ]:
day_of_week = []
for idx, day in df_all_corr.iterrows():
    day_of_week.append(idx.weekday())
    
df_all_corr['day_of_week'] = day_of_week

In [ ]:
f, ax = plt.subplots(figsize=(12, 12))
sns.corrplot(df_all_corr, annot=True, sig_stars=False,
             diag_names=False, ax=ax)

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
calendar = USFederalHolidayCalendar()

In [ ]:
df_all_corr['day_of_week'].unique()

In [ ]:
ref_col = 'all_day_to'

In [ ]:
list(df_all_corr.columns.values)

In [ ]:
features = ['Visibility',
 'DryBulbCelsius',
 'WindSpeed',
 'HourlyPrecip',
 'day_of_week']

In [ ]:
def numpy_arrays_for_tr_and_cv(features, df_T, df_CV, ref_column):
    X_T = df_T[features].values
    X_CV = df_CV[features].values
    y_T = df_T[ref_column].values
    y_CV = df_CV[ref_column].values
    return X_T, y_T, X_CV, y_CV

In [ ]:
X_T, y_T, X_CV, y_CV = numpy_arrays_for_tr_and_cv(features, df_all_corr, df_all_corr, ref_col)

In [ ]:
from sklearn import linear_model

#Define the model.
lin_regr = linear_model.LinearRegression()
fit1 = lin_regr.fit(X_T, y_T) 
df_all_corr['traffic_fit'] = fit1.predict(X_T)

In [ ]:
def plot_fitted_and_ref(fitted_data, df_H):
    plt.figure(figsize=(15,5))
    plt.title('Predicted and Reference Traffic Data', fontsize = 30)
    fitted_data.plot(color="r", marker = '.', label = 'fitted')
    plt.xlabel('Time', fontsize = 18)
    plt.ylabel('Number of Cars', fontsize = 18)
    plt.legend() 
    df_H['O3_ppb'].plot(label = 'reference')
    plt.show()

In [ ]:
df_all_corr['traffic_fit'].plot(marker = 'o')